# **OpenPose Point Extracting**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
base_dir='/content/gdrive/My Drive/'
base_dir_OpenPose = '/content/gdrive/My Drive/OpenPose/'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## **1. 라이브러리 호출**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle as pkl
import sklearn

import os
import cv2
import time

from matplotlib.patches import Rectangle
from PIL import Image

import csv  #csv 파일 저장

##**2. 이미지 프레임 캡쳐 후 디렉토리 저장**

In [ ]:
#구글링, 인용 원본 코드 출처: https://deftkang.tistory.com/182
#동영상에서 이미지 프레임 추출
vid_dir = "IMG_6305.MOV"           #비디오 지정

vid_name = vid_dir[:8]            #비디오별로 개별 저장을 위한 이름 슬라이싱
print(vid_name)
vidcap = cv2.VideoCapture(base_dir + vid_dir)

count = 0
frame_folder="OpenPose_frames"

os.makedirs(base_dir_OpenPose+frame_folder,exist_ok=True)

while(vidcap.isOpened()):
    ret, image = vidcap.read()

    # print(int(vidcap.get(1))
    if(int(vidcap.get(1)) % 20 == 0):   #20 프레임당 한개의 이미지
 
        #이미지프레임 저장
        if count <= 9:  #인덱싱 문제 때문에 반드시 10의 자리 이하는 앞에 0을 추가해야함, 사용할 프레임 수 현재: 16
            cv2.imwrite(base_dir_OpenPose + 'OpenPose_frames/OpenPose_frame0'+'%d.jpg' % count, image)
            print("OpenPose_frame0%d.jpg saved" % count)
        else:
            cv2.imwrite(base_dir_OpenPose +'OpenPose_frames/OpenPose_frame%d.jpg' % count, image)            
            print("OpenPose_frame%d.jpg' saved" % count)        
        count += 1
        
    if count == 16:   #16장 찍을 경우 종료
            vidcap.release()
            
#vidcap.release()      #강제 종료

IMG_6305


## **3. 캡쳐된 이미지들 인덱싱, 전처리 하기 위해 파일들 오름차순 정렬**

In [ ]:
img_path_list = []   #모든 이미지 경로를 저장한 리스트

global V             # 순수 파일명만 들어갈 리스트
V = []
global V_s           # 순수 파일명 임시 리스트
V_s = []

frame_dir = "OpenPose_frames"     #frame이 저장된 폴더, frame_dirs 과 다름!

root_dir = base_dir_OpenPose + frame_dir   #동영상에서 캡쳐된 프레임들

total_file_list = os.listdir(root_dir)       #파일 읽어온 다음 total_file_list 에 저장

def get_images():
    global img_path_list           #이미지들의 경로를 추출해서 리스트에 추가
    possible_img_extension = ['.jpg']

    for(root, dir, files) in os.walk(root_dir):
        print("디렉토리 내의 파일개수: " + str(len(files)))
        for i in range(0,len(files)):     
                if len(files) > 0:
                    for file_name in files: 
                        if file_name.endswith(str(i) + ".jpg"):                                             
                              img_path = root + '/' + file_name     
                              V_s.append(file_name)
                              img_path_list.append(img_path)
                    
get_images()  #함수 호출: 디렉토리 안에 있는 모든 이미지파일들만 추출


#중복값 제거 하고 다시 원래 리스트에 삽입
result = []
for value in img_path_list:
    if value not in result:
        result.append(value)
img_path_list = result  #다시 전역변수로 선언 (바깥쪽)
img_num = len(img_path_list)   #이미지 트래킹될 모든 파일들의 숫자, 전역변수
        
result = []        
for value in V_s:
    if value not in result:
        result.append(value)
V_s = result


#print("img_path_list: " + str(img_num))
print("사용될 예정의 이미지 파일 개수: " + str(len(V_s)))


#이미지 파일 경로 리스트 오름차순 정리      
img_path_list.sort()                      


#확장자 없이 파일명만 그대로 가져오기. 인용 원본코드출처:https://appia.tistory.com/502
for i in V_s:  
    if os.path.isdir(root_dir + r"/" + i):
        pass

    else :
        #print("file : " + i)
        if i.count(".") == 1 : # . 이 한개일떄

            V_split = i.split(".")
            #print("file Name : " + V_split[0])
            V.append(V_split[0])
        else :
            #print(len(i))
            for k in range(len(i)-1,0,-1): # . 이 여러개 일때
                if i[k] == ".":
                    #print("file Name : "+i[:k])
                    break

print("이미지 추출에 사용될 파일개수는: " + str(len(V)) + " 입니다.")

V.sort()   #파일명이 들어간 리스트, 번호대로 정렬
for i in range(len(V)):
    print(V[i])

디렉토리 내의 파일개수: 16
사용될 예정의 이미지 파일 개수: 16
이미지 추출에 사용될 파일개수는: 16 입니다.
OpenPose_frame00
OpenPose_frame01
OpenPose_frame02
OpenPose_frame03
OpenPose_frame04
OpenPose_frame05
OpenPose_frame06
OpenPose_frame07
OpenPose_frame08
OpenPose_frame09
OpenPose_frame10
OpenPose_frame11
OpenPose_frame12
OpenPose_frame13
OpenPose_frame14
OpenPose_frame15


## **4. 관절좌표 출력, 결측치 처리 등, 사용 될 데이터 프레임 형성**

In [ ]:
print("사용할 이미지 파일들의 개수: " + str(len(img_path_list)))
frames=len(V)

#Data Frame - 다만 디렉토리내의 개수에 따라 바뀜
nose_x = np.zeros(shape=(frames,))
nose_y = np.zeros(shape=(frames,))
left_shoulder_x = np.zeros(shape=(frames,))
left_shoulder_y = np.zeros(shape=(frames,))
right_shoulder_x = np.zeros(shape=(frames,))
right_shoulder_y = np.zeros(shape=(frames,))
left_elbow_x = np.zeros(shape=(frames,))
left_elbow_y = np.zeros(shape=(frames,))
right_elbow_x = np.zeros(shape=(frames,))
right_elbow_y = np.zeros(shape=(frames,))
left_wrist_x = np.zeros(shape=(frames,))
left_wrist_y = np.zeros(shape=(frames,))
right_wrist_x = np.zeros(shape=(frames,))
right_wrist_y = np.zeros(shape=(frames,))
left_hip_x = np.zeros(shape=(frames,))
left_hip_y = np.zeros(shape=(frames,))
right_hip_x = np.zeros(shape=(frames,))
right_hip_y = np.zeros(shape=(frames,))
left_knee_x = np.zeros(shape=(frames,))
left_knee_y = np.zeros(shape=(frames,))
right_knee_x = np.zeros(shape=(frames,))
right_knee_y = np.zeros(shape=(frames,))
left_ankle_x = np.zeros(shape=(frames,))
left_ankle_y = np.zeros(shape=(frames,))
right_ankle_x = np.zeros(shape=(frames,))
right_ankle_y = np.zeros(shape=(frames,))   

pose_name = []  #pose_name = data['type_info']['exercise']    #테스트 파일의 동작은 동작분류 학습모델에서 추론

사용할 이미지 파일들의 개수: 16


## **5. 신경망 구축**
* COCO Model 파일과 prototxt 파일로 신경망 구축진행
* 화면 출력을 위한 각 부위 연결 저장 리스트 형성

In [ ]:
'''
<COCO KEYPOINT DETECTION>
https://cocodataset.org/#keypoints-2018

<신체부위 인덱싱 원본 인용 코드>
https://hanryang1125.tistory.com/2
'''
#prototxt파일: 합성곱 신경망(CNN)에 대한 정의, 파라미터와 설정값들을 포함하는 .prototxt 파일을 사용합니다. (출처: https://kyubot.tistory.com/97)
#COCO에서 각 파트번호, 선으로 연결된 POSE_PAIRS, COCO모델에 적용시키기 위함
BODY_PARTS_COCO = {0: "Nose", 1: "Neck", 2: "RShoulder", 3: "RElbow", 4: "RWrist",
                   5: "LShoulder", 6: "LElbow", 7: "LWrist", 8: "RHip", 9: "RKnee",
                   10: "RAnkle", 11: "LHip", 12: "LKnee", 13: "LAnkle", 14: "REye",
                   15: "LEye", 16: "REar", 17: "LEar", 18: "Background"}

#사진상에 각각의 라인을 만들어 가시화 하기 위한 리스트 관절쌍 조합
POSE_PAIRS_COCO = [[0, 1], [0, 14], [0, 15], [1, 2], [1, 5], [1, 8], [1, 11], [2, 3], [3, 4],
                   [5, 6], [6, 7], [8, 9], [9, 10], [12, 13], [11, 12], [14, 16], [15, 17]]

# 신경 네트워크의 구조를 지정하는 두 가지 파일 (다양한 계층이 배열되는 방법 등) for COCO
Data_prototxt = base_dir+"pose_deploy_linevec.prototxt"   # 프로토텍스트 파일
Data_weights = base_dir+"pose_iter_440000.caffemodel"     # COCO Model의 caffemodel

##**6. 최종 출력 알고리즘**
* 각 프레임 이미지에 관절 좌표와 좌표들을 이은 선을 표시해줌
* 결측치 발생시 가장 근접한 좌표들을 통해 근사치를 얻어와서 새롭게 출력

In [ ]:
# 이미지 읽어오기
#image = cv2.imread("fitness_jpg_test/033-1-1-21-Z1_E-0000001.jpg") 
'''
OpenPose ToolKit, 원본 인용코드, 라이브러리 내용 참조 링크들:
<OpenPose 관련문서>
https://learnopencv.com/deep-learning-based-human-pose-estimation-using-opencv-cpp-python/

<OpenPose GitHub>
https://github.com/CMU-Perceptual-Computing-Lab/openpose

<Opencv DNN 모듈>
https://docs.opencv.org/4.5.1/d5/de7/tutorial_dnn_googlenet.html

<Blob 객체>
https://www.pyimagesearch.com/2017/11/06/deep-learning-opencvs-blobfromimage-works/

<Kaggle OpenPose - 딥러닝 진행할 caffemodel>
https://www.kaggle.com/changethetuneman/openpose-model/version/1?select=pose_iter_584000.caffemodel

<현재 셀 출력 파트 인용,참조 원본 코드 출처>
https://learnopencv.com/deep-learning-based-human-pose-estimation-using-opencv-cpp-python/
https://hanryang1125.tistory.com/2
https://m.blog.naver.com/rhrkdfus/221531159811
'''
# 네트워크 불러오기 - COCO Model과 prototxt 파일을 사용
net = cv2.dnn.readNetFromCaffe(Data_prototxt, Data_weights)

#임시 저장 변수들
row_list = []
err_printed_img = []
temp_list = []
temp_list_str = []
    
#************************************관절좌표 추출 시작 ***************************
for k in range(0,len(img_path_list)):
    ff = np.fromfile(img_path_list[k], np.uint8)
    image = cv2.imdecode(ff, cv2.IMREAD_UNCHANGED)
    
    points = [] #관절 좌표값을 담을 리스트

#     #신뢰도가 0.5 이상이면 물체가 거의 정확하게 감지되었음
#     #보통 임계값이 0에 가까울 수록 탐지되는 물체의 수가 많고 1에 가까울수록 정확도가 높아짐
    #----------COCO MODEL-------------------------------
    def output_keypoints(frame, Data_prototxt, Data_weights, threshold, model_name, BODY_PARTS):
        global points
        global temp_list


        # 입력 이미지의 사이즈 정의, dnn 모듈 네트워크에 사용
        image_height = 368
        image_width = 368

        # 네트워크에 넣기 위한 전처리 blob 객체로 저장
        inpBlob = cv2.dnn.blobFromImage(frame, 1.0 / 255, (image_width, image_height), (0, 0, 0), swapRB=False, crop=False)

        # 전처리된 blob 객체를 네트워크에 입력 
        # Pass the blob to the network
        net.setInput(inpBlob)

        # 결과 받아오기 - out 변수에 전달
        out = net.forward()        #Make forward pass
        out_height = out.shape[2]
        out_width = out.shape[3]   # The fourth dimension is the width of the output map.

        frame_height, frame_width = frame.shape[:2] # 원본 이미지의 높이, 너비를 받아오기

        # 포인트 리스트 초기화
        points = []

        print("************************************************************************************")
        print(f"현재 이미지 파일은 "+ img_path_list[k] + " 입니다." + str(k) + "번째 이미지 입니다.\n" )

        for i in range(len(BODY_PARTS)):

            #***********prob의 개념 매우 중요 ***************************
            #신뢰도가 0.5 이상이면 물체가 거의 정확하게 감지되었음
            #보통 임계값이 0에 가까울 수록 탐지되는 물체의 수가 많고 1에 가까울수록 정확도가 높아짐

            # 관절 좌표와 위치의 신뢰도 측정
            prob_map = out[0, i, :, :]

            # 최소값, 최대값, 최소값 위치, 최대값 위치
            min_val, prob, min_loc, point = cv2.minMaxLoc(prob_map)

            # 원본 이미지에 맞게 포인트 위치 조정
            x = (frame_width * point[0]) / out_width
            x = int(x)
            y = (frame_height * point[1]) / out_height
            y = int(y)

            #선: 노랑, 관절: 초록, 빨강숫자: 제대로 출력, 파랑: 결측치
            if prob > threshold:  # 임계값 이상일 경우 올바른 값, 관절좌표 출력 빨간색으로 출력
                cv2.circle(frame, (x, y), 7, (0, 255, 0), thickness=-1, lineType=cv2.FILLED)  #thickness=-1 내부 채워진 원
                cv2.putText(frame, str(i), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 1, lineType=cv2.LINE_AA)

                points.append((x, y))

            else:  #텍스트를 파랑색으로 출력 - 의미없는 값 사실상 좌표 출력 안된것
                cv2.circle(frame, (x, y), 7, (0, 255, 0), thickness=-1, lineType=cv2.FILLED)
                cv2.putText(frame, str(i), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 1, lineType=cv2.LINE_AA)

                points.append((0,0))


        #전처리 단계
        #*****************각 관절좌표들을 모아놓을 리스트들*********int로 저장********** str 좌표들은 따로 저장함****
        nose_x[k] = int(points[0][0])
        nose_y[k] = int(points[0][1])
        left_shoulder_x[k] = int(points[5][0])
        left_shoulder_y[k] = int(points[5][1])
        right_shoulder_x[k] = int(points[2][0])
        right_shoulder_y[k] = int(points[2][1])
        left_elbow_x[k] = int(points[6][0])
        left_elbow_y[k] = int(points[6][1])
        right_elbow_x[k] = int(points[3][0])
        right_elbow_y[k] = int(points[3][1])
        left_wrist_x[k] = int(points[7][0])
        left_wrist_y[k] = int(points[7][1])
        right_wrist_x[k] = int(points[4][0])
        right_wrist_y[k] = int(points[4][1])
        left_hip_x[k] = int(points[11][0])
        left_hip_y[k] = int(points[11][1])
        right_hip_x[k] = int(points[8][0])
        right_hip_y[k] = int(points[8][1])
        left_knee_x[k] = int(points[12][0])
        left_knee_y[k] = int(points[12][1])
        right_knee_x[k] = int(points[9][0])
        right_knee_y[k] = int(points[9][1])
        left_ankle_x[k] = int(points[13][0])
        left_ankle_y[k] = int(points[13][1])
        right_ankle_x[k] = int(points[10][0])
        right_ankle_y[k] = int(points[10][1])     
        
        temp_list = [nose_x[k], nose_y[k], left_shoulder_x[k],   #프레임별로 26개의 관절 값들을 저장합니다.
                     left_shoulder_y[k], right_shoulder_x[k],
                     right_shoulder_y[k], left_elbow_x[k],
                     left_elbow_y[k], right_elbow_x[k],
                     right_elbow_y[k], left_wrist_x[k],
                     left_wrist_y[k], right_wrist_x[k],
                     right_wrist_y[k], left_hip_x[k],
                     left_hip_y[k], right_hip_x[k],
                     right_hip_y[k], left_knee_x[k],
                     left_knee_y[k], right_knee_x[k],
                     right_knee_y[k], left_ankle_x[k], left_ankle_y[k], right_ankle_x[k], right_ankle_y[k]]
        
        row_list.append(temp_list)   #모든 프레임에 대해서 각 프레임마다 26개 관절 좌표값 리스트로 저장합니다.
        
        return frame

    def output_keypoints_with_lines(frame, POSE_PAIRS):  #라인을 그어주는 함수입니다.
        global temp_list
        print()
        for pair in POSE_PAIRS:
            part_a = pair[0]  # 0 (Head)
            part_b = pair[1]  # 1 (Neck)
            if points[part_a] and points[part_b]:   #두개의 좌표가 연결 가능할 경우 연결함 : 미리PAIRS로 조합된 리스트로 확인
      
                 cv2.line(frame, points[part_a], points[part_b], (100, 255, 255), 4) 
            else:
                 print("")


            #***********************디렉토리 생성 후 이미지 저장***************************
            if not os.path.exists(base_dir_OpenPose+'OpenPose_results'):
                os.makedirs(base_dir_OpenPose+'OpenPose_results')
            
            cv2.imwrite(base_dir_OpenPose+'OpenPose_results/'+ vid_name +'_printed' + str(k) +'.jpg', frame)



    # 이미지를 COCO모델에 대입
    frame_coco = image.copy()
    # COCO Model에 사용하기 위해 인자들 입력 1.이미지프레임, 2.프로텍스트, 3.머신러닝 weight, 4.임계값
    frame_COCO = output_keypoints(frame=frame_coco, Data_prototxt=Data_prototxt, Data_weights=Data_weights,
                                 threshold=0.01, model_name="COCO", BODY_PARTS=BODY_PARTS_COCO)
    output_keypoints_with_lines(frame=frame_COCO, POSE_PAIRS=POSE_PAIRS_COCO)    


    #정수 형태로 리스트를 다시 저장함
    row_int = [nose_x, nose_y, left_shoulder_x, left_shoulder_y, right_shoulder_x, right_shoulder_y, left_elbow_x, left_elbow_y, 
          right_elbow_x, right_elbow_y, left_wrist_x, left_wrist_y, right_wrist_x, right_wrist_y, left_hip_x, left_hip_y,
          right_hip_x, right_hip_y, left_knee_x, left_knee_y, right_knee_x, right_knee_y, left_ankle_x, left_ankle_y, 
          right_ankle_x, right_ankle_y]
    
print("반복문 완료")
print("*****************-----###########전체 관절 좌표 출력 끝##################------***************")

#데이터프레임 생성용도의 col_name
col_name_OpenPose = ['nose_x', 'nose_y', 'left_shoulder_x', 'left_shoulder_y', 'right_shoulder_x', 'right_shoulder_y', 'left_elbow_x', 
     'left_elbow_y', 'right_elbow_x', 'right_elbow_y', 'left_wrist_x', 'left_wrist_y', 'right_wrist_x', 
     'right_wrist_y', 'left_hip_x', 'left_hip_y', 'right_hip_x', 'right_hip_y', 'left_knee_x', 'left_knee_y', 'right_knee_x', 
     'right_knee_y', 'left_ankle_x', 'left_ankle_y', 'right_ankle_x', 'right_ankle_y']

final_row_list = []    #2차원 리스트. 총 16개의 프레임에 각각 26개의 관절좌표값이 저장되어있음
final_row_list = row_int

########################### 결측치 처리 알고리즘 ###################################
last_idx = len(final_row_list[0])-1
for i in range(0,len(final_row_list)):   # nose_x, nose_y, left_shoulder_x, ... 관절부위들  
    for j in range(last_idx):   # 해당 관절부위의 시계열 데이터 탐색     
        if final_row_list[i][j] == 0:  #만약 단 한개의 좌표라도 0이면 그 좌표값을 근사치로 바꿔준다.
            #1                      
            if final_row_list[i][0] == 0:   #만약 첫번째 좌표가 0이다.
                
                #하지만, 만약 현재좌표 다음 좌표값이 0일경우 그 다음으로 넘어가기위함. j부터 시작
                for k in range(j,last_idx): #추가 인덱스.                     
                    if final_row_list[i][k] != 0:                        
                        final_row_list[i][j] = final_row_list[i][k]
                        break
            #2
            elif final_row_list[i][last_idx] == 0:  #만약 마지막 좌표가 0이다.
                #결측값이 아니며 마지막 좌표에서 가까운 이전 좌표값을 가져온다.
                for k in range(last_idx + 1):
                    if final_row_list[i][last_idx - k] != 0:
                        final_row_list[i][last_idx] = final_row_list[i][last_idx - k]
                        break
            #3
            else:       #나머지들은 직전과 직후 값의 평균
                   #하지만, 만약 현재좌표 다음 좌표값이 0일경우 그 다음으로 넘어가기위함. j부터 시작
                for k in range(j,last_idx): #추가 인덱스. 마지막까지 감
                    if final_row_list[i][k] != 0:
                        final_row_list[i][j] = (final_row_list[i][j-1] + final_row_list[i][k]) / 2                   
                        break

######################################################################################

for i in range(len(final_row_list)):    #스트링
  for j in range(len(final_row_list[i])):
    final_row_list[i][j] = int(final_row_list[i][j])


nose_x = final_row_list[0]
nose_y = final_row_list[1]
left_shoulder_x = final_row_list[2]
left_shoulder_y = final_row_list[3]
right_shoulder_x = final_row_list[4]
right_shoulder_y = final_row_list[5]
left_elbow_x = final_row_list[6]
left_elbow_y = final_row_list[7]
right_elbow_x = final_row_list[8]
right_elbow_y = final_row_list[9]
left_wrist_x = final_row_list[10]
left_wrist_y = final_row_list[11]
right_wrist_x = final_row_list[12]
right_wrist_y = final_row_list[13]
left_hip_x = final_row_list[14]
left_hip_y = final_row_list[15]
right_hip_x = final_row_list[16]
right_hip_y = final_row_list[17]
left_knee_x = final_row_list[18]
left_knee_y = final_row_list[19]
right_knee_x = final_row_list[20]
right_knee_y = final_row_list[21]
left_ankle_x = final_row_list[22]
left_ankle_y = final_row_list[23]
right_ankle_x = final_row_list[24]
right_ankle_y = final_row_list[25]

print("**********각 프레임별 관절좌표 저장 완료************")

### 데이터 프레임 생략


************************************************************************************
현재 이미지 파일은 /content/gdrive/My Drive/OpenPose/OpenPose_frames/OpenPose_frame00.jpg 입니다.0번째 이미지 입니다.


************************************************************************************
현재 이미지 파일은 /content/gdrive/My Drive/OpenPose/OpenPose_frames/OpenPose_frame01.jpg 입니다.1번째 이미지 입니다.


************************************************************************************
현재 이미지 파일은 /content/gdrive/My Drive/OpenPose/OpenPose_frames/OpenPose_frame02.jpg 입니다.2번째 이미지 입니다.


************************************************************************************
현재 이미지 파일은 /content/gdrive/My Drive/OpenPose/OpenPose_frames/OpenPose_frame03.jpg 입니다.3번째 이미지 입니다.


************************************************************************************
현재 이미지 파일은 /content/gdrive/My Drive/OpenPose/OpenPose_frames/OpenPose_frame04.jpg 입니다.4번째 이미지 입니다.


**********************************************************************